###Gerekli kütüphaneler import edilir

In [ ]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

### 1. Veri İndirme

#####Veri kümesini curl kullanarak yerel sürücü düğümünün /tmp klasörüne indirin

In [ ]:
%sh curl -Lb /tmp/gcokie "https://drive.google.com/uc?export=download&id=1APm8CqNE0iGvtEtKWW8jl6PUWTwm-had" -o "/tmp/abcnews-date-text.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0

  7 52.2M    7 4176k    0     0   939k      0  0:00:56  0:00:04  0:00:52  939k
 61 52.2M   61 32.0M    0     0  6105k      0  0:00:08  0:00:05  0:00:03 30.1M
100 52.2M  100 52.2M    0     0  9157k      0  0:00:05  0:00:05 --:--:-- 34.5M


In [ ]:
%fs ls file:/tmp/

path,name,size,modificationTime
file:/tmp/hsperfdata_root/,hsperfdata_root/,4096,1702265803489
file:/tmp/chauffeur-daemon.pid,chauffeur-daemon.pid,4,1702265787285
file:/tmp/chauffeur-daemon-params,chauffeur-daemon-params,22,1702265787113
file:/tmp/.ICE-unix/,.ICE-unix/,4096,1702265778373
file:/tmp/driver-env.sh,driver-env.sh,3321,1702265800725
file:/tmp/.XIM-unix/,.XIM-unix/,4096,1702265778373
file:/tmp/systemd-private-780f26eb614449dbad5495ec22c61bcd-ntp.service-BPtcfi/,systemd-private-780f26eb614449dbad5495ec22c61bcd-ntp.service-BPtcfi/,4096,1702265778457
file:/tmp/python_lsp_logs/,python_lsp_logs/,4096,1702265885214
file:/tmp/custom-spark.conf,custom-spark.conf,727,1702265785513
file:/tmp/abcnews-date-text.csv,abcnews-date-text.csv,54763207,1702266260196


#####Dosyayı yerel sürücü düğümünün dosya sisteminden DBFS'ye taşıyın

In [ ]:
dbutils.fs.mv("file:/tmp/abcnews-date-text.csv", "dbfs:/datasets/abcnews-date-text.csv")

Out[3]: True

In [ ]:
%fs ls /datasets/

path,name,size,modificationTime
dbfs:/datasets/abcnews-date-text.csv,abcnews-date-text.csv,54763207,1702266265000
dbfs:/datasets/songsDataset.csv,songsDataset.csv,15856091,1682176681000


#####Veri kümesi dosyasını bir Spark Dataframe'i olarak okuyun

In [ ]:
news_df = spark.read.load("dbfs:/datasets/abcnews-date-text.csv",
                         format="csv",
                         sep=",",
                         inferSchema="true",
                         header="true"
                         )

#####Yüklenen veri kümesinin satır ve sütun sayısını kontrol edin

In [ ]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(news_df.count(), len(news_df.columns)))

The shape of the dataset is 1093281 rows by 2 columns


#####Yüklenen veri kümesinin şemasını yazdırın

In [ ]:
news_df.printSchema()

root
 |-- publish_date: integer (nullable = true)
 |-- headline_text: string (nullable = true)



#####Veri kümesinin ilk 5 satırını görüntüleme

In [ ]:
news_df.show(5, truncate=False)

+------------+--------------------------------------------------+
|publish_date|headline_text                                     |
+------------+--------------------------------------------------+
|20030219    |aba decides against community broadcasting licence|
|20030219    |act fire witnesses must be aware of defamation    |
|20030219    |a g calls for infrastructure protection summit    |
|20030219    |air nz staff in aust strike for pay rise          |
|20030219    |air nz strike to affect australian travellers     |
+------------+--------------------------------------------------+
only showing top 5 rows



##### Yinelenen haberlerin sayısını (varsa) hesaplama

In [ ]:
print("The total number of duplicated news are {:d} out of {:d}".
      format(news_df.count() - news_df.dropDuplicates(['headline_text']).count(), news_df.count()))

The total number of duplicated news are 27320 out of 1093281


##### En çok tekrar eden ilk 10 haberi görüntüleyin

In [ ]:
news_df.groupby(["headline_text"]).count().sort("count", ascending=False).show(10)

+--------------------+-----+
|       headline_text|count|
+--------------------+-----+
|           abc sport|  755|
|         abc weather|  736|
| national rural news|  648|
|abc business news...|  588|
|          closer am1|  315|
|   abc entertainment|  315|
|          closer pm1|  307|
|abc entertainment...|  240|
|       kohler report|  206|
|           closer pm|  200|
+--------------------+-----+
only showing top 10 rows



##### Yinelenen haberleri kaldırma

In [ ]:
news_df = news_df.dropDuplicates(["headline_text"])

In [ ]:
print("The total number of unique news is: {:d}".format(news_df.count()))

The total number of unique news is: 1065961


##### Headline_text sütunu boyunca herhangi bir eksik değer (NULL) olup olmadığını kontrol etme

In [ ]:
news_df.where(col("headline_text").isNull()).count()
# Alternatively, using filter:
# news_df.filter(news_df.headline_text.isNull()).count()

Out[12]: 0

### 2. Veri Ön İşleme

Bu örnekte, metin verileriyle çalışıyoruz ve nihai hedefimiz, bildiğimiz kümeleme algoritmalarından birini (örneğin, K-means) kullanarak haberleri tutarlı "konu" gruplarına ayırmaktır. Bu, doğal dil işleme (NLP) olarak adlandırılan daha genel bir alanın özel bir problemidir.


###### Herhangi bir NLP görevinin ön adımları olarak, önce en azından aşağıdaki ardışık düzen yürütülmelidir:

<br>

- Metin temizleme:
 - Büyük/küçük harf normalleştirme (<code>lower</code>) -> tüm metni küçük harfe çevir;
 - Baştaki ve sondaki boşluklarını filtreleyin (<code>trim</code>);
 - Noktalama işaretlerini filtreleyin (<code>regexp_replace</code>);
 - Yukarıdaki adımdan kaynaklanan dahili fazladan boşlukları filtreleyin (<code>regexp_replace</code> + <code>trim</code>).
- Belirteçleştirme (<code>Tokenizer</code>): genellikle sınırlayıcı olarak boşluk kullanarak, ham metni tek tek _belirteçler_ (yani kelimeler) listesine bölme
- Gereksiz kelimelerin kaldırılması (<code>StopWordsRemover</code>): _stopwords_ denilen, yani ingilizcede "the", "a", "me" gibi metnin derin anlamına katkıda bulunmayan kelimelerin kaldırılması.
- Stemming (<code>SnowballStemmer</code>): Her kelimeyi köküne veya tabanına indirgeme. Örneğin, "balıkçılık", "balıklar", "balıkçı", hepsi "balık" köküne indirgenir.

In [ ]:
def clean_text(df, column_name="headline_text"):
    """
    This function takes the raw text data and applies a standard NLP preprocessing pipeline consisting of the following steps:
      - Text cleaning
      - Tokenization
      - Stopwords removal
      - Stemming (Snowball stemmer)

    parameter: dataframe
    returns: the input dataframe along with the `cleaned_content` column as the results of the NLP preprocessing pipeline

    """
    from pyspark.sql.functions import udf, col, lower, trim, regexp_replace
    from pyspark.ml.feature import Tokenizer, StopWordsRemover
    from nltk.stem.snowball import SnowballStemmer # BE SURE NLTK IS INSTALLED ON THE CLUSTER USING THE "LIBRARIES" TAB IN THE MENU

    # Text preprocessing pipeline
    print("***** Text Preprocessing Pipeline *****\n")

    # 1. Text cleaning
    print("# 1. Text Cleaning\n")
    # 1.a Case normalization
    print("1.a Case normalization:")
    lower_case_news_df = df.select(lower(col(column_name)).alias(column_name))
    lower_case_news_df.show(10)
    # 1.b Trimming
    print("1.b Trimming:")
    trimmed_news_df = lower_case_news_df.select(trim(col(column_name)).alias(column_name))
    trimmed_news_df.show(10)
    # 1.c Filter out punctuation symbols
    print("1.c Filter out punctuation:")
    no_punct_news_df = trimmed_news_df.select((regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name))
    no_punct_news_df.show(10)
    # 1.d Filter out any internal extra whitespace
    print("1.d Filter out extra whitespaces:")
    cleaned_news_df = no_punct_news_df.select(trim(regexp_replace(col(column_name), " +", " ")).alias(column_name))
    cleaned_news_df.show(10)

    # 2. Tokenization (split text into tokens)
    print("# 2. Tokenization:")
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_news_df).cache()
    tokens_df.show(10)

    # 3. Stopwords removal
    print("# 3. Stopwords removal:")
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    terms_df = stopwords_remover.transform(tokens_df).cache()
    terms_df.show(10)

    # 4. Stemming (Snowball stemmer)
    print("# 4. Stemming:")
    stemmer = SnowballStemmer(language="english")
    stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
    terms_stemmed_df = terms_df.withColumn("terms_stemmed", stemmer_udf("terms")).cache()
    terms_stemmed_df.show(10)

    return terms_stemmed_df

In [ ]:
clean_news_df = clean_text(news_df)

***** Text Preprocessing Pipeline *****

# 1. Text Cleaning

1.a Case normalization:
+--------------------+
|       headline_text|
+--------------------+
|ambitious olsson ...|
|a g calls for inf...|
|antic delighted w...|
|aba decides again...|
|act fire witnesse...|
|aust addresses un...|
|aussie qualifier ...|
|air nz staff in a...|
|australia is lock...|
|air nz strike to ...|
+--------------------+
only showing top 10 rows

1.b Trimming:
+--------------------+
|       headline_text|
+--------------------+
|ambitious olsson ...|
|a g calls for inf...|
|antic delighted w...|
|aba decides again...|
|act fire witnesse...|
|aust addresses un...|
|aussie qualifier ...|
|air nz staff in a...|
|australia is lock...|
|air nz strike to ...|
+--------------------+
only showing top 10 rows

1.c Filter out punctuation:
+--------------------+
|       headline_text|
+--------------------+
|ambitious olsson ...|
|a g calls for inf...|
|antic delighted w...|
|aba decides again...|
|act fire witnes

### **Feature Engineering**

Makine öğrenimi teknikleri doğrudan metin verileri üzerinde çalışamaz; Aslında, kelimeler önce makine öğrenimi algoritmalarının kullanabileceği bazı sayısal temsillere dönüştürülmelidir. Bu süreç genellikle _vektörleştirme_ olarak bilinir.

Vektörleştirme açısından, bunun yalnızca tek bir kelimeyi tek bir sayıya dönüştürmek olmadığını hatırlamak önemlidir. Kelimeler sayılara dönüştürülebilirken, bir belgenin tamamı bir vektöre çevrilebilir. Ayrıca, metin verilerinden türetilen vektörler genellikle yüksek boyutludur. Bunun nedeni, özellik uzayının her boyutunun bir kelimeye karşılık gelmesi ve belgelerdeki dilin binlerce kelimeye sahip olabilmesidir.

## TF-IDF
Bilgi almada, **tf-idf** - terimin kısaltması frekans-ters belge frekansı - bir kelimenin bir koleksiyondaki veya tümcedeki bir belge için ne kadar önemli olduğunu yansıtması amaçlanan sayısal bir istatistiktir.

tf-idf değeri, bir sözcüğün belgede görünme sayısıyla orantılı olarak artar ve bazı sözcüklerin genel olarak daha sık göründüğü gerçeğini düzeltmeye yardımcı olan, tümcedeki sözcüğün sıklığıyla dengelenir.

In [ ]:
RANDOM_SEED = 42 # used below to run the actual K-means clustering
VOCAB_SIZE = 1000 # number of words to be retained as vocabulary
MIN_DOC_FREQ = 10 # minimum number of documents a word has to appear in to be included in the vocabulary
N_GRAMS = 2 # number of n-grams (if needed)
N_FEATURES = 200 # default embedding vector size (if HashingTF or, later, Word2Vec are used)

In [ ]:
def extract_tfidf_features(df, column_name="terms_stemmed"):
    """
    This fucntion takes the text data and converts it into a term frequency-inverse document frequency vector

    parameter: dataframe
    returns: dataframe with tf-idf vectors

    """

    # Importing the feature transformation classes for doing TF-IDF
    from pyspark.ml.feature import HashingTF, CountVectorizer, IDF, NGram
    from pyspark.ml import Pipeline

    ## Extracting n-grams from text
    #ngrams = NGram(n=N_GRAMS, inputCol=column_name, outputCol="ngrams")
    #ngrams.transform(df)

    ## Creating Term Frequency Vector for each word
    #cv = CountVectorizer(inputCol=column_name, outputCol="tf_features", vocabSize=VOCAB_SIZE, minDF=MIN_DOC_FREQ)
    #cv_model = cv.fit(df)
    #tf_features_df = cv_model.transform(df).cache()

    ## Alternatively to CountVectorizer, use HashingTF
    #hashing_TF = HashingTF(inputCol=column_name, outputCol="tf_features", numFeatures=N_FEATURES)
    #tf_features_df = hashing_TF.transform(df).cache()

    ## Carrying out Inverse Document Frequency on the TF data
    #idf = IDF(inputCol="tf_features", outputCol="features")
    #idf_model = idf.fit(tf_features_df)
    #tf_idf_features_df = idf_model.transform(tf_features_df).cache()

    # USING PIPELINE
    #ngrams = NGram(n=N_GRAMS, inputCol=column_name, outputCol="ngrams")
    cv = CountVectorizer(inputCol=column_name, outputCol="tf_features", vocabSize=VOCAB_SIZE, minDF=MIN_DOC_FREQ)
    # hashingTF = HashingTF(inputCol=column_name, outputCol="tf_features", numFeatures=N_FEATURES)
    idf = IDF(inputCol="tf_features", outputCol="features")

    pipeline = Pipeline(stages=[cv, idf]) # add `ngrams` and replace `cv` with `hashingTF`, if needed
    features = pipeline.fit(df)
    tf_idf_features_df = features.transform(df).cache()

    return tf_idf_features_df

In [ ]:
features = extract_tfidf_features(clean_news_df)

In [ ]:
features.select(col("features")).show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------+
|features                                                                                                               |
+-----------------------------------------------------------------------------------------------------------------------+
|(1000,[119,237,661],[5.102449590618002,5.518614963648062,6.354827009507638])                                           |
|(1000,[4,208,772],[3.8823192696353392,5.4334762463817246,6.509787515266589])                                           |
|(1000,[475,537],[6.058948720530817,6.158926541193276])                                                                 |
|(1000,[0,3,29,300],[3.4168850753934796,3.789504107309776,4.549932382926653,5.7145934315482325])                        |
|(1000,[7,24,358,769],[4.065169151015052,4.464557201001815,5.82804730250002,6.502880109466778])                         |
|(1000,[36,49,116],[4.64

In [ ]:
clean_news_df.show(5, truncate=False)

+---------------------------------------------+--------------------------------------------------------+----------------------------------------------+------------------------------------------+
|headline_text                                |tokens                                                  |terms                                         |terms_stemmed                             |
+---------------------------------------------+--------------------------------------------------------+----------------------------------------------+------------------------------------------+
|heavy metal deposits survey nearing end      |[heavy, metal, deposits, survey, nearing, end]          |[heavy, metal, deposits, survey, nearing, end]|[heavi, metal, deposit, survey, near, end]|
|planning for intersection revamp begins      |[planning, for, intersection, revamp, begins]           |[planning, intersection, revamp, begins]      |[plan, intersect, revamp, begin]          |
|blix orders destruction 

###### Olası sıfır uzunluklu vektörleri kontrol edin ve kaldırın

In [ ]:
@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

In [ ]:
print("Total n. of zero-length vectors: {:d}".
      format(features.where(num_nonzeros("features") == 0).count()))

Total n. of zero-length vectors: 28341


In [ ]:
features = features.where(num_nonzeros("features") > 0)

In [ ]:
print("Total n. of zero-length vectors (after removal): {:d}".
      format(features.where(num_nonzeros("features") == 0).count()))

Total n. of zero-length vectors (after removal): 0


### 3. K-means Clustering

In [ ]:
N_CLUSTERS = 10 # number of output clusters (K)
DISTANCE_MEASURE = "euclidean" # alternatively, "cosine"
MAX_ITERATIONS = 100 # maximum number of iterations of K-means EM algorithm
TOLERANCE = 0.000001 # tolerance between consecutive centroid updates (i.e., another stopping criterion)

In [ ]:
def k_means(dataset,
            n_clusters,
            distance_measure=DISTANCE_MEASURE,
            max_iter=MAX_ITERATIONS,
            tol=TOLERANCE,
            features_col="features",
            prediction_col="cluster",
            random_seed=RANDOM_SEED):

  from pyspark.ml.clustering import KMeans

  print("""Training K-means clustering using the following parameters:
  - K (n. of clusters) = {:d}
  - max_iter (max n. of iterations) = {:d}
  - distance measure = {:s}
  - random seed = {:d}
  """.format(n_clusters, max_iter, distance_measure, random_seed))
  # Train a K-means model
  kmeans = KMeans(featuresCol=features_col,
                   predictionCol=prediction_col,
                   k=n_clusters,
                   initMode="k-means||",
                   initSteps=5,
                   tol=tol,
                   maxIter=max_iter,
                   seed=random_seed,
                   distanceMeasure=distance_measure)
  model = kmeans.fit(dataset)

  # Make clusters
  clusters_df = model.transform(dataset).cache()

  return model, clusters_df

In [ ]:
model, clusters_df = k_means(features, N_CLUSTERS, max_iter=MAX_ITERATIONS, distance_measure=DISTANCE_MEASURE)

Training K-means clustering using the following parameters: 
  - K (n. of clusters) = 10
  - max_iter (max n. of iterations) = 100
  - distance measure = euclidean
  - random seed = 42
  


##### Elde edilen kümeleri değerlendirmek için kullanılan fonksiyon (Silhouette Coefficient)

In [ ]:


def evaluate_k_means(clusters,
                     metric_name="silhouette",
                     distance_measure="squaredEuclidean", # cosine
                     prediction_col="cluster"
                     ):

  from pyspark.ml.evaluation import ClusteringEvaluator

  # Evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator(metricName=metric_name,
                                  distanceMeasure=distance_measure,
                                  predictionCol=prediction_col
                                  )

  return evaluator.evaluate(clusters)



######Kümelemeyi değerlendirin (Silhouette Coefficient)

In [ ]:
evaluate_k_means(clusters_df, distance_measure="squaredEuclidean")

Out[28]: 0.03925454725124379

In [ ]:
clusters_df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+
|       headline_text|              tokens|               terms|       terms_stemmed|         tf_features|            features|cluster|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+
|heavy metal depos...|[heavy, metal, de...|[heavy, metal, de...|[heavi, metal, de...|(1000,[119,237,66...|(1000,[119,237,66...|      0|
|planning for inte...|[planning, for, i...|[planning, inters...|[plan, intersect,...|(1000,[4,208,772]...|(1000,[4,208,772]...|      0|
|blix orders destr...|[blix, orders, de...|[blix, orders, de...|[blix, order, des...|(1000,[475,537],[...|(1000,[475,537],[...|      0|
|wa opp says polic...|[wa, opp, says, p...|[wa, opp, says, p...|[wa, opp, say, po...|(1000,[0,3,29,300...|(1000,[0,3,29,300...|      0|
|forum calls for m...|[forum, calls, fo...|[foru

In [ ]:
clusters_df.groupBy("cluster").count().sort("cluster").show()

+-------+------+
|cluster| count|
+-------+------+
|      0|973389|
|      1|  7407|
|      2| 10434|
|      3|  3466|
|      4|  5248|
|      5|  4259|
|      6| 11223|
|      7|  3726|
|      8| 13861|
|      9|  4607|
+-------+------+



In [ ]:
# Get unique values in the grouping column
clusters = sorted([x[0] for x in clusters_df.select("cluster").distinct().collect()])
print("Cluster IDs: [{:s}]".format(", ".join([str(c) for c in clusters])))

# Create a filtered DataFrame for each group in a list comprehension
cluster_list = [clusters_df.where(clusters_df.cluster == x) for x in clusters]

# Show the results (first 5 cluters)
for x_id, x in enumerate(cluster_list):
  print("Showing the first 10 records of cluster ID #{:d}".format(x_id))
  x.select(["cluster", "headline_text"]).show(10, truncate=True)

Cluster IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Showing the first 10 records of cluster ID #0
+-------+--------------------+
|cluster|       headline_text|
+-------+--------------------+
|      0|heavy metal depos...|
|      0|planning for inte...|
|      0|blix orders destr...|
|      0|wa opp says polic...|
|      0|forum calls for m...|
|      0|environment centr...|
|      0|man missing after...|
|      0|gonzalez in the w...|
|      0|chinese economy f...|
|      0|un inspectors set...|
+-------+--------------------+
only showing top 10 rows

Showing the first 10 records of cluster ID #1
+-------+--------------------+
|cluster|       headline_text|
+-------+--------------------+
|      1|govt warns austra...|
|      1|workplace safety ...|
|      1|unbeaten freitas ...|
|      1|sydney water elec...|
|      1|us senators talk ...|
|      1|opposition worrie...|
|      1|safety reminder i...|
|      1|level crossing sa...|
|      1|skipping periods ...|
|      1|mine unions back ...|
+

In [ ]:
EMBEDDING_SIZE = 150 # size of embedding Word2Vec vectors

In [ ]:
def extract_w2v_features(df, column_name="terms"):
  from pyspark.ml.feature import Word2Vec

  word2vec = Word2Vec(vectorSize=EMBEDDING_SIZE, minCount=5, inputCol=column_name, outputCol="features", seed=RANDOM_SEED)
  model = word2vec.fit(df)
  features = model.transform(df).cache()

  return model, features

In [ ]:
model, w2v_features = extract_w2v_features(clean_news_df)

In [ ]:
w2v_features.show(truncate=False)

+--------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
vecs = model.getVectors()
syms = model.findSynonyms("doctor", 2)
syms.show()

+-------+------------------+
|   word|        similarity|
+-------+------------------+
|  nurse|  0.70589679479599|
|surgeon|0.6926254034042358|
+-------+------------------+



In [ ]:
K_MIN = 2
K_MAX = 18
STEP = 2

In [ ]:
def elbow_method(data, k_min=K_MIN, k_max=K_MAX, step=STEP, max_iter=MAX_ITERATIONS, distance_measure=DISTANCE_MEASURE):
  results = []
  for k in range(k_min, k_max, step):
    model, clusters_df = k_means(data, k, max_iter=max_iter, distance_measure=distance_measure)
    results.append([k, model.summary.trainingCost])

  return pd.DataFrame(results, columns = ['K', 'SSE'])

In [ ]:
# Get results from elbow method
elbow_results = elbow_method(w2v_features)

Training K-means clustering using the following parameters: 
  - K (n. of clusters) = 2
  - max_iter (max n. of iterations) = 100
  - distance measure = euclidean
  - random seed = 42
  
Training K-means clustering using the following parameters: 
  - K (n. of clusters) = 4
  - max_iter (max n. of iterations) = 100
  - distance measure = euclidean
  - random seed = 42
  
Training K-means clustering using the following parameters: 
  - K (n. of clusters) = 6
  - max_iter (max n. of iterations) = 100
  - distance measure = euclidean
  - random seed = 42
  
